In [2]:
import numpy as np
import cv2 as cv
import argparse

def main():
    parser = argparse.ArgumentParser(description='Lucas-Kanade Optical Flow')
    cap = cv.VideoCapture(video_path)

    if not cap.isOpened():
        print('Error: Unable to open video file.')
        return

    # Parameters for Shi-Tomasi corner detection
    feature_params = dict(
        maxCorners=100,
        qualityLevel=0.3,
        minDistance=7,
        blockSize=7
    )

    # Parameters for Lucas-Kanade optical flow
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)
    )

    # Create some random colors for visualization
    color = np.random.randint(0, 255, (100, 3))

    # Read the first frame and find corners in it
    ret, old_frame = cap.read()
    if not ret:
        print('Error: No frames grabbed!')
        return

    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
    p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)

    while True:
        ret, frame = cap.read()
        if not ret:
            print('End of video.')
            break

        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # Calculate optical flow
        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
            frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

        img = cv.add(frame, mask)
        cv.imshow('Optical Flow', img)

        k = cv.waitKey(30) & 0xff
        if k == 27:
            break

        # Update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    # Release video capture and close all windows
    cap.release()
    cv.destroyAllWindows()

if __name__ == "__main__":
    video_path = '/Users/kuldeeprajgour/Desktop/Optical-Flow-Motion/cars_on_highway (1080p).mp4'
    main(video_path)



usage: ipykernel_launcher.py [-h] image
ipykernel_launcher.py: error: the following arguments are required: image


SystemExit: 2

/Users/kuldeeprajgour/anaconda3/envs/tensorflow/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
